In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.
        builder.
        appName("curso_pyspark").
        getOrCreate()
)

spark

# Básico de Pyspark

## Dataframes

<!-- ###
### create table (
###    nome VARCHAR(100) not null,
###    sobrenome VARCHAR(100) not null,
###    idade INT not null
###)

#  nome sobrenome idade
# (N1      N2       I1) -->

create table ( <br />
   nome VARCHAR(100) not null, <br />
   sobrenome VARCHAR(100) not null, <br />
   idade INT not null <br />
) <br />
<br /><br />
nome sobrenome idade
(N1      N2       I1)

In [ ]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

data = [
    ##  C1          C2        C3
    ("Matheus", "Cantarutti", 31),
    ("Ana", "Cláudia", 18),
    ("Brunno", "Oliveira", 25)
]

schema = StructType([
    StructField("Nome", StringType(), True),
    StructField("Sobre_Nome", StringType(), True),
    StructField("Idade", IntegerType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()

df.show()

In [ ]:
df.createOrReplaceTempView("pessoas")

In [ ]:
### SQL --> Pyspark

spark.sql( 
'''    
    select
        *
    from pessoas
    where Idade < 20
'''
).show()

In [ ]:
from pyspark.sql import functions as F

# df.filter('Idade < 20').show()
df.filter(
    F.col('Idade') < 20
).show()

## Tipo de Dados

- TIPO TEXTO/STRING >> abrangendo apenas as funções que tratam texto
- TIPO DATA (DATA ESTÁ COM O TIPO DE STRING) >> Converter o seu texto para Data

- FLOAT/DECIMAL e INTERGER

In [ ]:
spark.sql('''
    select
        *,
        cast(Idade * 5 as string) as Idade_2
    from pessoas
''').printSchema()

In [ ]:
(   
    # nome da coluna, expressões/funcao
    df.withColumn('Idade_2', F.col('Idade') * 5)
      .withColumn('data', F.lit('2025-01-01')) # current date
      .withColumn('data2', F.to_date(F.col('data'), 'yyyy-MM-dd'))
      .withColumn('Idade_3', F.expr('cast(Idade * 5 as string) as Idade_3'))

).show()

## Cardinalidade

- Aula teórica explicativa sobre cardinalidade

## Dataframe de exemplo

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType

spark = (
    SparkSession.
        builder.
        getOrCreate()
)
# CLOUD --> spark.sql('select * from bd.aulas.tabelas')
# df <- na leitura dos arquivos (.csv ou xlsx)
## spark.read

data = [
    ("1", "PAGO"),
    ("2", "APROVADO"),
    ("3", "RECUSADO"),
    ("4", "ENTREGUE"),
    ("5", "CANCELADO"),
    ("6", "NÃO ENTREGUE")
]

schema = StructType([
    StructField("cd_identificacao", StringType(), True),
    StructField("status", StringType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()
df.show()

# Lendo arquivos

## Command Separated Value (.csv) 

In [ ]:
caminho = './dados/csv/'
cliente = 'clientes.csv'
status = 'status.csv'
pedidos = 'pedidos.csv'

clientes = (
    spark.read.csv(
        f'{caminho}{cliente}', 
        sep=';',
        header=True
    )
)

status = (
    spark.read.csv(
        f'{caminho}{status}', 
        sep=';',
        header=True
    )
)

pedidos = (
    spark.read.csv(
        f'{caminho}{pedidos}', 
        sep=';',
        header=True
    )
)

pedidos.show()

## Excel

In [ ]:
import pandas as pd

caminho = './dados/xlsx/'
cliente = 'clientes.xlsx'
aba = 'clientes'

def ler_excel(file_path, aba):
    try:
        df = pd.read_excel(file_path, sheet_name=aba, engine='openpyxl')
    except FileNotFoundError:
        print(f"Arquivo {file_path} não encontrado.")
    except ValueError as e:
        print(f"Erro ao ler a aba de nome {aba}: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
    return df

df = ler_excel(f'{caminho}{cliente}', aba)
df = spark.createDataFrame(df)
print(type(df))

# Intermediário

## Filtros

### Filter

In [ ]:
from pyspark.sql import functions as F

In [ ]:
caminho = './dados/csv/'
cliente = 'clientes.csv'

clientes = (
    spark.read.csv(
        f'{caminho}{cliente}', 
        sep=';',
        header=True
    )
)

clientes.show()
clientes.createOrReplaceTempView("clientes")

In [ ]:
spark.sql('''
    select 
        * 
    from clientes
    where sexo = 'F'
''').show()

In [ ]:
clientes2 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            F.col('sexo') == 'F'
        )
)
clientes2.show()

In [ ]:
clientes3 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            F.col('sexo') == 'M'
        )
)
clientes3.show()

In [ ]:
clientes4 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            ~ (F.col('sexo') == 'F')
        )
)
clientes4.show()

### Isin

In [ ]:
spark.sql('''
    select
        *
    from clientes
    where cd_cliente in ('2', '3')
''').show()

In [ ]:
spark.sql('''
    select
        *
    from clientes
    where cd_cliente not in ('2', '3')
''').show()

In [ ]:
clientes.filter(
    F.col('cd_cliente').isin(['2', '3'])
).show()

In [ ]:
clientes.filter(
    ~(F.col('cd_cliente').isin(['2', '3']))
).show()

## Tratamentos

### Strings para Números

### regex com Pyspark

In [ ]:
caminho = './dados/csv/'
pedidos = 'pedidos.csv'

pedidos = (
    spark.read.csv(
        f'{caminho}{pedidos}', 
        sep=';',
        header=True
    )
)

pedidos.show()
pedidos.createOrReplaceTempView("pedidos")

In [ ]:
pedidos.printSchema()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, DoubleType

pedidos2 = (
    # MacOS é diferente do Windows
    pedidos.withColumn('valor_limpo', F.regexp_replace(F.col('valor'),  r'R\$\s*', ''))
           .withColumn('valor_limpo', F.regexp_replace(F.col('valor_limpo'), r'\.', ''))
           .withColumn('valor_limpo', F.regexp_replace(F.col('valor_limpo'), r',', '.'))
           .withColumn('valor_limpo', F.col('valor_limpo').cast(DoubleType()))
)

pedidos2.show()

In [ ]:
pedidos2.printSchema()

In [ ]:
pedidos2.groupBy('cd_cliente').agg(
    F.mean('valor_limpo').alias('valor_mean_limpo'),
    F.sum('valor_limpo').alias('valor_sum_limpo'),
    F.max('valor_limpo').alias('valor_max_limpo'),
    F.min('valor_limpo').alias('valor_min_limpo'),
    F.avg('valor_limpo').alias('valor_avg_limpo'),
    F.median('valor_limpo').alias('valor_median_limpo'),
    F.mode('valor_limpo').alias('valor_mode_limpo')
).show()

### Trabalhando com Datas

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.
        builder.
        appName("curso_pyspark").
        getOrCreate()
)

spark

In [ ]:
caminho = './dados/csv/'
clientes = 'clientes.csv'

clientes = (
    spark.read.csv(
        f'{caminho}{clientes}', 
        sep=';',
        header=True
    )
)

clientes.show()
clientes.createOrReplaceTempView("clientes")

In [ ]:
clientes3 = (
    clientes
        .select("cd_cliente", "data_nascimento")
        .withColumn("data_nascimento2", F.to_date(
            F.col("data_nascimento"), "yyyy-MM-dd"
        ))
        .withColumn("ano", F.year(F.col("data_nascimento2")))
        .withColumn("mes", F.month(F.col("data_nascimento2")))
        .withColumn("dia", F.day(F.col("data_nascimento2")))
        
        .withColumn(
            "data_BR", 
                F.concat(
                    F.col("dia"), # dia
                        F.lit("/"), 
                    F.col("mes"), # mes
                        F.lit("/"), 
                    F.col("ano") # ano
                )
        )
        .withColumn("data_BR", F.try_to_timestamp(
            F.col("data_BR"), "dd/MM/yyyy"
        ))

)

clientes3.show()

In [ ]:
# pode substituir a forma pela qual a conversão de data ocorre.
        .withColumn(
            "data_BR_str",
            F.concat(
                F.lpad(F.col("dia"), 2, "0"), F.lit("/"),
                F.lpad(F.col("mes"), 2, "0"), F.lit("/"),
                F.col("ano")
            )
        )
        .withColumn("data_BR", F.to_date(F.col("data_BR_str"), "dd/MM/yyyy"))

## Um pouco mais de Modelagem

In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

caminho = './dados/csv/'
pedidos = 'pedidos.csv'
clientes = 'clientes.csv'
status = 'status.csv'

pedidos = (
    spark.read.csv(
        f'{caminho}{pedidos}', 
        sep=';',
        header=True
    )
)

clientes = (
    spark.read.csv(
        f'{caminho}{clientes}', 
        sep=';',
        header=True
    )
)

status = (
    spark.read.csv(
        f'{caminho}{status}', 
        sep=';',
        header=True
    )
)


In [ ]:
pedidos.groupBy('cd_cliente').agg(
    F.count('cd_cliente').alias('count')
).orderBy(
    F.col('count').desc()
).show()

In [ ]:
pedidos.groupBy('cd_cliente').count().orderBy('count', ascending=False).show()

# Avançado

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

spark = (
    SparkSession.
        builder.
        appName("curso_pyspark").
        getOrCreate()
)

spark

In [ ]:
def ler_arquivo(caminho, nome_arquivo):
    arquivo = (
        spark.read.csv(
            f'{caminho}{nome_arquivo}',
            sep=';',
            header=True
        )
    )
    return arquivo

pedidos = ler_arquivo('./dados/csv/', 'pedidos.csv')

pedidos.show()

## Join - Movimentando informações entre tabelas

### Busca Horizontal

### 'Busca' vertical

## Condicionais com When

## Pivot tables
### Transformando visões